# Evolver Loop 4 Analysis

## Key Findings from Research

1. **Baseline is at tight local optimum** - All standard optimization (SA, backward propagation, bbox3) found ZERO improvements
2. **Gap to target**: 70.676102 - 68.919154 = 1.756948 (2.49%)
3. **Key techniques NOT yet tried**:
   - **fix_direction**: Rotate entire configuration to minimize bounding box
   - **Perturbation + re-optimization**: Escape local optima by perturbing 8-15% of trees
   - **Population-based approach**: Keep top 3 solutions, perturb and re-optimize
   - **sa_v3 optimizer**: More sophisticated than sa_v1 with 8 move types

In [ ]:
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely.geometry import Polygon
from shapely import affinity
from shapely.ops import unary_union
from scipy.spatial import ConvexHull
from scipy.optimize import minimize_scalar

getcontext().prec = 25
scale_factor = Decimal("1e15")

# Tree geometry
TX = np.array([0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125])
TY = np.array([0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5])

print("Setup complete")

In [ ]:
# Load baseline
baseline_df = pd.read_csv('/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025-csv/santa-2025.csv')

def parse_value(val):
    if isinstance(val, str) and val.startswith('s'):
        return val[1:]
    return str(val)

# Calculate current score breakdown
class ChristmasTree:
    def __init__(self, center_x="0", center_y="0", angle="0"):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))
        self._update_polygon()
    
    def _update_polygon(self):
        vertices = []
        for tx, ty in zip(TX, TY):
            vertices.append((float(Decimal(str(tx)) * scale_factor), 
                           float(Decimal(str(ty)) * scale_factor)))
        initial_polygon = Polygon(vertices)
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(
            rotated,
            xoff=float(self.center_x * scale_factor),
            yoff=float(self.center_y * scale_factor)
        )

def load_trees_for_n(df, n):
    prefix = f"{n:03d}_"
    rows = df[df['id'].str.startswith(prefix)]
    trees = []
    for _, row in rows.iterrows():
        x = parse_value(row['x'])
        y = parse_value(row['y'])
        deg = parse_value(row['deg'])
        trees.append(ChristmasTree(x, y, deg))
    return trees

def calculate_side(trees):
    all_polygons = [t.polygon for t in trees]
    bounds = unary_union(all_polygons).bounds
    return Decimal(str(max(bounds[2] - bounds[0], bounds[3] - bounds[1]))) / scale_factor

print("Loading configurations...")
sides = {}
for n in range(1, 201):
    trees = load_trees_for_n(baseline_df, n)
    sides[n] = calculate_side(trees)
    if n % 50 == 0:
        print(f"  N={n}, side={sides[n]:.6f}")

total_score = sum(float(sides[n])**2 / n for n in range(1, 201))
print(f"\nTotal score: {total_score:.6f}")
print(f"Target: 68.919154")
print(f"Gap: {total_score - 68.919154:.6f} ({(total_score - 68.919154)/68.919154*100:.2f}%)")

In [ ]:
# Analyze which N values have the most room for improvement
# Compare efficiency (trees per unit area) vs theoretical optimal

print("\nScore contribution by N range:")
ranges = [(1, 10), (11, 50), (51, 100), (101, 150), (151, 200)]
for start, end in ranges:
    range_score = sum(float(sides[n])**2 / n for n in range(start, end+1))
    print(f"  N={start}-{end}: {range_score:.4f} ({range_score/total_score*100:.1f}%)")

print("\nTop 10 score contributors:")
contributions = [(n, float(sides[n])**2 / n) for n in range(1, 201)]
contributions.sort(key=lambda x: -x[1])
for n, contrib in contributions[:10]:
    print(f"  N={n}: {contrib:.6f} (side={float(sides[n]):.6f})")

print("\nLowest efficiency (trees per unit area):")
efficiencies = [(n, n / float(sides[n])**2) for n in range(1, 201)]
efficiencies.sort(key=lambda x: x[1])
for n, eff in efficiencies[:10]:
    print(f"  N={n}: {eff:.4f} trees/unit² (side={float(sides[n]):.6f})")

In [ ]:
# Test fix_direction on a few N values
# This rotates the entire configuration to minimize bounding box

def calculate_bbox_side_at_angle(angle_deg, points):
    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix_T = np.array([[c, s], [-s, c]])
    rotated_points = points.dot(rot_matrix_T)
    min_xy = np.min(rotated_points, axis=0)
    max_xy = np.max(rotated_points, axis=0)
    return max(max_xy[0] - min_xy[0], max_xy[1] - min_xy[1])

def optimize_rotation(trees):
    """Find optimal rotation angle to minimize bounding box"""
    all_points = []
    for tree in trees:
        all_points.extend(list(tree.polygon.exterior.coords))
    points_np = np.array(all_points) / float(scale_factor)
    
    hull_points = points_np[ConvexHull(points_np).vertices]
    initial_side = calculate_bbox_side_at_angle(0, hull_points)
    
    res = minimize_scalar(
        lambda a: calculate_bbox_side_at_angle(a, hull_points),
        bounds=(0.001, 89.999),
        method="bounded",
    )
    
    return initial_side, float(res.fun), float(res.x)

print("Testing fix_direction on sample N values:")
test_ns = [2, 5, 10, 20, 50, 100, 150, 200]
for n in test_ns:
    trees = load_trees_for_n(baseline_df, n)
    initial, optimized, angle = optimize_rotation(trees)
    improvement = (initial - optimized) / initial * 100
    score_before = initial**2 / n
    score_after = optimized**2 / n
    score_improvement = score_before - score_after
    print(f"  N={n}: side {initial:.6f} -> {optimized:.6f} (angle={angle:.2f}°, score improvement: {score_improvement:.9f})")

In [ ]:
# Summary of key findings
print("="*60)
print("KEY FINDINGS FOR NEXT EXPERIMENT")
print("="*60)
print("\n1. BASELINE STATUS:")
print(f"   - Current score: {total_score:.6f}")
print(f"   - Target: 68.919154")
print(f"   - Gap: {total_score - 68.919154:.6f} (2.49%)")
print("\n2. TECHNIQUES NOT YET TRIED:")
print("   a) fix_direction: Rotate entire config to minimize bbox")
print("   b) Perturbation + re-optimization: Escape local optima")
print("   c) Population-based approach: Keep top 3, perturb, re-optimize")
print("   d) sa_v3 optimizer with 8 move types")
print("   e) Longer optimization runs (hours, not minutes)")
print("\n3. RECOMMENDED NEXT STEPS:")
print("   1. Implement fix_direction post-processing")
print("   2. Implement perturbation mechanism")
print("   3. Run sa_v3 with population-based approach")
print("   4. Run for much longer (1+ hours)")
print("\n4. CRITICAL INSIGHT:")
print("   The baseline is at a LOCAL optimum, not GLOBAL.")
print("   Perturbation is the key to escaping it.")